In [ ]:
! pip install datasets
! pip install evaluate
! pip install accelerate -U
! pip install transformers torch peft
! pip install -U accelerate

In [ ]:
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer, RobertaForMultipleChoice, get_scheduler
import torch
import random
import os
# from transformers import TrainingArguments
# from transformers import Trainer
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np
from tqdm.auto import tqdm
from peft import get_peft_model, LoraConfig, TaskType, LoraModel
from torch.utils.data import DataLoader

# sam's import
from peft import LoraConfig, get_peft_model
# !pip install accelerate -U
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import os
import numpy as np
from transformers import get_scheduler
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn import functional as F
import json
from torch.nn import CrossEntropyLoss

# Roberta-base
Instead of directly trying to guess the word, our first attemp is to generate three other choices from the original dataset, in addtion to the answers, and then we will fine tune Roberta-base to choose the right answer out of the four answers. We just want to prove that the model can capture some information from the input sentences.

In [ ]:
# @title Mount google
# Change this path to your project location on drive
# if you are running locally, then I assume that the relative path of the project directory is ../
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

# Tehila's path
# drive_path = '/content/drive/MyDrive/huji/NLP/Projects/CryptoniteAnalysis/'

# sam's path
drive_path = '/content/drive/MyDrive/Projects/CryptoniteAnalysis/'

os.chdir(drive_path)

import_dir = 'Baselines/MultipleChoices/Models/roberta-base/checkpoint-14500'


Mounted at /content/drive/


In [ ]:
# @title Generative model from multi-choice

from transformers import RobertaForMultipleChoice, RobertaConfig, RobertaForCausalLM

# Load the pre-trained multiple-choice model
# model_name = "path_to_your_roberta_multiple_choice_model"
# multiple_choice_model = RobertaForMultipleChoice.from_pretrained(model_name)

# Extract the underlying RoBERTa model
config = RobertaConfig.from_pretrained(import_dir)
config.is_decoder = True  # Set the model to be a decoder
config.add_cross_attention = False  # If you need cross-attention

# Create a new model with a generative head
generative_model = RobertaForCausalLM(config)

trained_model = RobertaForMultipleChoice.from_pretrained(import_dir)
print(trained_model)
# Load the weights from the multiple-choice model to the new generative model
multiple_choice_state_dict = trained_model.roberta.state_dict()

# Filter out the weights related to cross-attention and pooler
filtered_state_dict = {k: v for k, v in multiple_choice_state_dict.items() if "crossattention" not in k and "pooler" not in k}

# Load the filtered state dict
missing_keys, unexpected_keys = generative_model.roberta.load_state_dict(filtered_state_dict, strict=False)

# Print missing and unexpected keys
print("Missing keys:", missing_keys)
print(f"Unexpected keys:{unexpected_keys}\n\n" )

print(generative_model)

# Save the generative model
# generative_model.save_pretrained("path_to_save_generative_model")

RobertaForMultipleChoice(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [ ]:
# @title Step 1: Read the JSONL File

import json

def load_jsonl(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    return data

dataset_path= 'Baselines/MultipleChoices/ProcessedDatasets/'
train_data = load_jsonl(dataset_path+"cryptonite-train-choice.jsonl")
test_data = load_jsonl(dataset_path+"cryptonite-test-choice.jsonl")
eval_data = load_jsonl(dataset_path+"cryptonite-val-choice.jsonl")


In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# generative_model.to(device)

In [ ]:
# # @title compute_metrics
# import numpy as np
# from datasets import load_metric

# # Load accuracy metric
# metric = load_metric("accuracy")

# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     print("hi")
#     return metric.compute(predictions=predictions, references=labels)

In [ ]:
# @title GPT train lora - load data


In [ ]:
# def print_maxs(examples):
#   max_clue_len = 0
#   max_answer_len = 0
#   for example in examples:
#     if len(example['clue']) > max_clue_len:
#       max_clue_len = len(example['clue'])
#       if(len(example['clue'])) > 120:
#         print("clue: "+example['clue'])
#     if len(example['answer']) > max_answer_len:
#       max_answer_len = len(example['answer'])
#       if(len(example['answer'])) > 20:
#         print("clue: "+example['clue']+"\n\tanswer: "+example['answer'])
#   print(f"max clue len = {max_clue_len}")
#   print(f"max answer len = {max_answer_len}")

# print_maxs(dataset['train'])
# print_maxs(dataset['validation'])

In [ ]:
# @title Training functions
RECORD_HYPERPARAMETERS = 'record_hyperparameters'
RECORD_TRAIN_LOSS_AND_ACCURACY = 'record_train_loss_and_accuracy'
RECORD_TEST_LOSS_AND_ACCURACY = 'record_test_loss_and_accuracy'
RECORD_MODEL = 'record_model'

def save_lora_parameters(model, output_dir):
    '''
    Notice that we are also saving the unfreeze parameters here in addition to LoRA parameters.
    This function will vary from model to model: because the unfreezed parameters are different.
    Somehow for T5 and BART, their head are both called 'lm_head'.
    '''
    lora_params = {k: v for k, v in model.named_parameters() if ('lora' in k or 'lm_head' in k)}
    torch.save(lora_params, os.path.join(output_dir, "lora_params.pth"))


def write_results(output_dir, result_type, results, **kwargs):
    # write the hyper parameter under output_dir
    if result_type == RECORD_HYPERPARAMETERS:
        hyper_parameters_file = os.path.join(output_dir, 'hyper_parameters.json')
        with open(hyper_parameters_file, 'w') as f:
            json.dump(results, f)
            return
    # write the train and validate results under output_dir
    if result_type == RECORD_TRAIN_LOSS_AND_ACCURACY:
        json_file = os.path.join(output_dir,f'train_metrics.json')
        if os.path.exists(json_file):
            with open(json_file, 'r') as file:
                data = json.load(file)
        else:
            # Initialize data as an empty dictionary or appropriate structure
            data = []

        # Append the results to the existing data
        data.append(results)
        with open(json_file, 'w') as f:
            json.dump(data, f, indent=4)
            return

    # write the validate and test results under output_dir
    if result_type == RECORD_TEST_LOSS_AND_ACCURACY:
        json_file = os.path.join(output_dir, f'validate_and_test_metrics.json')
        if os.path.exists(json_file):
            with open(json_file, 'r') as file:
                data = json.load(file)
        else:
            # Initialize data as an empty dictionary or appropriate structure
            data = []

        # Append the results to the existing data
        data.append(results)
        with open(json_file, 'w') as f:
            json.dump(data, f, indent=4)
            return

    # store the model under output_dir (result is a model)
    if result_type == RECORD_MODEL:
        model = results
        save_lora_parameters(model, output_dir)
        return


EVAL_SUBSAMPLE_SIZE = 1000
VAL_SET_SIZE = 10000
TEST_SET_SIZE = 10000

EVAL_PER_STEP = 4800

def calculate_accuracy(logits, labels, tokenizer):
    '''
    There can be two way to calculate accuracy:
    1. compare what percentage of the output tokens are the same (expect for special tokens)
    If we want to compare number of tokens to be the same, then we can flatten the tokens and compare one by one.
    predictions = predictions.view(-1)
    labels = labels.view(-1)

    2. compare how many answers are correct in a batch
    Then if we want to compare answers, the dumb way is to first batch decode them, and then compare the decoded strings oe by one.
    Another way is to compare the tokens without decoding. But I am not sure how to deal with special tokens (sometimes it might not
    Generate correct end tokens.)

    '''
    predictions = torch.argmax(logits, dim=-1)

    pred_words = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    gold_standard_words = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # calculate correct predictions
    correct_labels, total_labels = 0, 0
    for i in range(len(pred_words)):
        if pred_words[i] == gold_standard_words[i]:
            correct_labels += 1
        total_labels += 1
    accuracy = correct_labels / total_labels
    return accuracy

def customize_loss_and_accuracy(outputs, target, tokenizer):
    '''
    Potential bugs
    The output of the model is by default the same length as the input sequence.
    Here by chance the input sequence is the same length as the labels (I padded them to be 40)
    If we want to be bug free for calculate accuracy, then we need to consider situations when the input sequence is not the same length to the labels.

    After we batch_decode in accuracy, the evaluation time is super high. So I suggest we don't calculate accuracy during training.
    And also I feel like we don't need to evaluate during training -- take too much time.
    '''
    # make the input and target the correct size (input is (batch* seq_len, dictionary_size), output is (batch*seq_len))
    loss = F.cross_entropy(input=outputs.logits.view(-1, outputs.logits.size(-1)), target=target.view(-1))
    accuracy = calculate_accuracy(logits=outputs.logits, labels=target, tokenizer=tokenizer)
    return loss, accuracy

def train_batch(model, tokenizer, epoch, step, batch, device, optimizer, scheduler, epoch_dir):
    # set model to train mode
    model.train()

    # put everything on the right device
    batch =  {k: v.to(device) for k, v in batch.items()}
    batch_size = batch['labels'].shape[0]

    # clear gradients, same old as usual
    optimizer.zero_grad()

    # forward pass, T5 forced us to pass in labels, this is good for "teacher forcing" according to ChatGPT.
    outputs = model(**batch)

    # outputs.loss might be problematic because of the NllLossBackward0 without softmax, should use nn.CrossEntropy
    loss, accuracy = customize_loss_and_accuracy(outputs, target=batch['labels'], tokenizer=tokenizer)

    # back propagation
    loss.backward()
    optimizer.step()

    # scheduler adjust lr
    scheduler.step()

    # record the train loss and accuracy
    record = {"evaluate_set": 'train', "epoch":epoch, "batch":step,
              "avg_loss":loss.item()/batch_size, "accuracy":accuracy, 'subsample_size':"None"}
    print(record)
    # WRITE: save the result for this epoch
    write_results(epoch_dir, result_type=RECORD_TRAIN_LOSS_AND_ACCURACY, results=record)

    return


def evaluate_model(model, tokenizer, epoch, step, dataloaders, device, subsample_size, evaluate_set, epoch_dir):
    '''evaluate means validate or test'''
    # set model to eval mode
    model.eval()
    # calculate number of samples being evaluated
    total_validated_samples = 0
    # calculate total loss and total number of correct labels (weighted acuracy)
    total_loss = 0
    total_accurate = 0
    # turn off grad computation
    with torch.no_grad():
        # evaluate batch by batch
        for batch in dataloaders[evaluate_set]:
            # terminate the process if we are subsampling
            if total_validated_samples > subsample_size:
                break

            # put everything on the right device
            batch =  {k: v.to(device) for k, v in batch.items()}
            batch_size = batch['labels'].shape[0]

            # forward pass in the model
            outputs = model(**batch)

            # accumulate loss and accuracy
            loss, accuracy = customize_loss_and_accuracy(outputs, target=batch['labels'], tokenizer=tokenizer)
            total_loss += loss.item()
            total_accurate += accuracy * batch_size
            total_validated_samples += batch_size


    # calculate the loss and accuracy
    average_loss = total_loss/total_validated_samples
    accuracy = total_accurate/total_validated_samples

    # record the loss and accuracy
    record = {"evaluate_set": evaluate_set, "epoch":epoch, "batch":step,
              "avg_loss": average_loss, 'accuracy': accuracy, 'subsample_size': subsample_size}
    print(record)

    # write to file
    write_results(epoch_dir, result_type=RECORD_TEST_LOSS_AND_ACCURACY, results=record)
    return


def train_epoch(model, tokenizer, epoch, dataloaders, device, optimizer, scheduler, epoch_dir):
    # prepare output dir
    if not os.path.exists(epoch_dir):
        os.makedirs(epoch_dir)

    # evaluate at the beginning of the training
    evaluate_model(model=model, tokenizer=tokenizer, epoch=epoch, step=0, dataloaders=dataloaders, device=device, subsample_size=VAL_SET_SIZE, evaluate_set='validation', epoch_dir=epoch_dir)
    evaluate_model(model=model, tokenizer=tokenizer, epoch=epoch, step=0, dataloaders=dataloaders, device=device, subsample_size=TEST_SET_SIZE, evaluate_set='test', epoch_dir=epoch_dir)

    for step, batch in enumerate(dataloaders['train']):
        # train the batch
        train_batch(model, tokenizer, epoch, step, batch, device, optimizer, scheduler, epoch_dir)

        # validate the model once every 100 steps
        if step % EVAL_PER_STEP == 0:
            evaluate_model(model, tokenizer, epoch, step, dataloaders, device, subsample_size=EVAL_SUBSAMPLE_SIZE, evaluate_set='validation', epoch_dir=epoch_dir)

    # save the model
    write_results(epoch_dir, result_type=RECORD_MODEL, results=model)
    return


def train_model(model, tokenizer, output_dir, dataloaders, optimizer, scheduler, device, hyper_parameters):
    # get device
    model.to(device)

    # number of epochs
    num_train_epochs = hyper_parameters['num_train_epochs']

    # Create the subdirectory for the hyperparameters: this directory is where we will save the result of trainning
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # WRITE hyperparameter to subdirectory
    write_results(output_dir, result_type=RECORD_HYPERPARAMETERS, results=hyper_parameters)


    # train the model on the hyper parameters
    for epoch in range(num_train_epochs):
        epoch_dir = os.path.join(output_dir, f"epoch={epoch}")
        train_epoch(model, tokenizer, epoch, dataloaders, device, optimizer, scheduler, epoch_dir)

    # final evaluation
    evaluate_model(model=model, tokenizer=tokenizer, epoch=num_train_epochs, step="STOP", dataloaders=dataloaders, device=device, subsample_size=VAL_SET_SIZE, evaluate_set='validation', epoch_dir=epoch_dir)
    evaluate_model(model=model, tokenizer=tokenizer, epoch=num_train_epochs, step="STOP", dataloaders=dataloaders, device=device, subsample_size=TEST_SET_SIZE, evaluate_set='test', epoch_dir=epoch_dir)

    return

In [ ]:
from datasets import load_dataset
from transformers import RobertaTokenizer, DataCollatorForSeq2Seq, TrainingArguments, Trainer, RobertaForCausalLM

# generative_model.to(device)
# Load the dataset from JSON Lines files
dataset = load_dataset('json', data_files={'train': dataset_path+"cryptonite-train-choice.jsonl", 'validation': dataset_path+"cryptonite-val-choice.jsonl", 'test': dataset_path + 'cryptonite-test-choice.jsonl'})


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:

# Select the first 1000 examples from the train and validation datasets
dataset['train'] = dataset['train'].select(range(48000))
dataset['validation'] = dataset['validation'].select(range(VAL_SET_SIZE))
dataset['test'] = dataset['test'].select(range(TEST_SET_SIZE))
# Load the tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Tokenize the dataset
def tokenize_function(examples):

    inputs = tokenizer(examples['clue'], max_length=136, truncation=True, padding='max_length', return_tensors="pt")
    targets = tokenizer(examples['answer'], max_length=136, truncation=True, padding='max_length', return_tensors="pt")

    return {
        'input_ids': inputs['input_ids'].squeeze(),  # Remove batch dimension
        'attention_mask': inputs['attention_mask'].squeeze(),  # Remove batch dimension
        'labels': targets['input_ids'].squeeze()  # Remove batch dimension
    }

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['publisher','date', 'author', 'number', 'orientation', 'clue', 'answer','enumeration', 'quick','sub_publisher', 'choice1','choice2','choice3'])
tokenized_datasets.set_format("torch")

# Define LoRA configuration
lora_config_dict = {
    "r": 8,
    "lora_alpha": 32,
    "lora_dropout": 0.1,
    "target_modules": ["query", "value"],
    # unfreeze the head of the model too
    'modules_to_save': ['lm_head']
}
lora_config = LoraConfig(**lora_config_dict)

# Apply LoRA to the model
lora_model = get_peft_model(generative_model, lora_config)

# define hyperparameters
per_device_train_batch_size = 16
learning_rate = 5e-03
num_train_epochs = 5

# define the best hyper parameter
hyper_parameters = {
            'learning_rate': learning_rate,
            'per_device_train_batch_size': per_device_train_batch_size,
            'num_train_epochs': num_train_epochs
        }
hyper_parameters['lora_config'] = lora_config_dict


# initialize dataloaders
dataloaders = {}
dataloaders['train'] = DataLoader(tokenized_datasets['train'], batch_size=per_device_train_batch_size, shuffle=True)
dataloaders['test'] = DataLoader(tokenized_datasets['test'], batch_size=per_device_train_batch_size)
dataloaders['validation'] = DataLoader(tokenized_datasets['validation'], batch_size=per_device_train_batch_size, shuffle=True)  # shuffle because we want to subsample

# initialize optimizer: Notice: change the model here!
optimizer = optim.Adam(lora_model.parameters(), lr=learning_rate)

# initialize scheduler
# calculate the arguments for shceduler: it depends on the sample size, batch size and epochs
num_training_steps = len(tokenized_datasets['train'])
lr_scheduler_type = 'linear'
lr_scheduler_kwargs = {'optimizer':optimizer,
                        'num_warmup_steps':int(0.1 * num_training_steps),
                        'num_training_steps':int((num_training_steps/per_device_train_batch_size) * num_train_epochs)}
scheduler = get_scheduler(lr_scheduler_type, **lr_scheduler_kwargs)


# defining the output directory
model_fp = 'Roberta_MC2GEN'
output_dir = f'Baselines/MultiChoiceToGen/Results/{model_fp}/'

# define training parameters
training_parameters = {
    'model': lora_model,
    'tokenizer': tokenizer,
    'output_dir': output_dir,
    'dataloaders': dataloaders,
    'optimizer':optimizer,
    'scheduler':scheduler,
    'device':torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    'hyper_parameters':hyper_parameters
}

# train the model
train_model(**training_parameters)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/48000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Streaming output truncated to the last 5000 lines.
{'evaluate_set': 'train', 'epoch': 3, 'batch': 1005, 'avg_loss': 0.010597573593258858, 'accuracy': 0.0, 'subsample_size': 'None'}
{'evaluate_set': 'train', 'epoch': 3, 'batch': 1006, 'avg_loss': 0.009989991784095764, 'accuracy': 0.0, 'subsample_size': 'None'}
{'evaluate_set': 'train', 'epoch': 3, 'batch': 1007, 'avg_loss': 0.01105620339512825, 'accuracy': 0.0, 'subsample_size': 'None'}
{'evaluate_set': 'train', 'epoch': 3, 'batch': 1008, 'avg_loss': 0.010759043507277966, 'accuracy': 0.0, 'subsample_size': 'None'}
{'evaluate_set': 'train', 'epoch': 3, 'batch': 1009, 'avg_loss': 0.010838781483471394, 'accuracy': 0.0, 'subsample_size': 'None'}
{'evaluate_set': 'train', 'epoch': 3, 'batch': 1010, 'avg_loss': 0.01160985417664051, 'accuracy': 0.0, 'subsample_size': 'None'}
{'evaluate_set': 'train', 'epoch': 3, 'batch': 1011, 'avg_loss': 0.010369238443672657, 'accuracy': 0.0, 'subsample_size': 'None'}
{'evaluate_set': 'train', 'epoch': 3, 'ba

In [ ]:
tokenized_dataset_fp = 'Baselines/MultiChoiceToGen/ProcessedDatasets/'
tokenized_datasets.save_to_disk(tokenized_dataset_fp)

Saving the dataset (0/1 shards):   0%|          | 0/48000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

# Legacy

In [ ]:
# # @title GPT train lora - tokenize and train

# # Select the first 1000 examples from the train and validation datasets
# dataset['train'] = dataset['train'].select(range(5))
# dataset['validation'] = dataset['validation'].select(range(5))
# # Load the tokenizer
# tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# # Tokenize the dataset
# def tokenize_function(examples):

#     inputs = tokenizer(examples['clue'], max_length=136, truncation=True, padding='max_length', return_tensors="pt")
#     targets = tokenizer(examples['answer'], max_length=30, truncation=True, padding='max_length', return_tensors="pt")

#     return {
#         'input_ids': inputs['input_ids'].squeeze(),  # Remove batch dimension
#         'attention_mask': inputs['attention_mask'].squeeze(),  # Remove batch dimension
#         'labels': targets['input_ids'].squeeze()  # Remove batch dimension
#     }

# tokenized_datasets = dataset.map(tokenize_function, batched=True)
# tokenized_datasets = tokenized_datasets.remove_columns(['publisher','date', 'author', 'number', 'orientation', 'clue', 'answer','enumeration', 'quick','sub_publisher', 'choice1','choice2','choice3'])

# # Define LoRA configuration
# lora_config = LoraConfig(
#     r=8,
#     lora_alpha=32,
#     lora_dropout=0.1,
#     target_modules=["query", "value"]
# )

# # Apply LoRA to the model
# lora_model = get_peft_model(generative_model, lora_config)

# # Prepare the data collator
# data_collator = DataCollatorForSeq2Seq(
#     tokenizer=tokenizer, model=lora_model,
#     max_length=136,
#     padding='max_length',
# )

# # Set up training arguments
# lora_training_args = TrainingArguments(
#     output_dir="./results",
#     # evaluation_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     remove_unused_columns=False,

#     auto_find_batch_size=True,
#     gradient_accumulation_steps=4,
#     warmup_steps=2,
#     max_steps=40,
#     evaluation_strategy = 'steps',
#     eval_accumulation_steps = 1,
#     eval_steps = 10,
# )

# # Initialize the trainer
# lora_trainer = Trainer(
#     model=lora_model,
#     args=lora_training_args,
#     train_dataset=tokenized_datasets["train"],
#     eval_dataset=tokenized_datasets["validation"],
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
# )


# # @title GPT evaluate trained lora
# lora_eval_results = lora_trainer.evaluate()

# # Print evaluation results
# print("Evaluation results before training:", lora_eval_results)


# # Train the model
# lora_trainer.train()

# # Save the fine-tuned model
# lora_model.save_pretrained("MultiChoiceToGen/lora_1")
# tokenizer.save_pretrained("MultiChoiceToGen/token_1")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


Evaluation results before training: {'eval_model_preparation_time': 0.0242, 'eval_runtime': 1.2587, 'eval_samples_per_second': 3.972, 'eval_steps_per_second': 1.589}


Step,Training Loss,Validation Loss,Model Preparation Time
10,No log,No log,0.024200
20,No log,No log,0.024200
30,No log,No log,0.024200
40,No log,No log,0.024200


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
# # @title GPT evaluate trained lora
# lora_eval_results = lora_trainer.evaluate()

# # Print evaluation results
# print("Evaluation results:", lora_eval_results)

In [ ]:
# # @title Step 4: Post-process the Output

# def decode_answers(outputs):
#     answers = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
#     return answers

# # train_answers = decode_answers(train_outputs)
# test_answers = decode_answers(test_outputs)
# eval_answers = decode_answers(eval_outputs)

# print(test_answers)
# print(eval_answers)

In [ ]:
# # @title Step 5: Evaluate the Model

# def evaluate(clues, generated_answers, true_data):
#     for clue, gen_answer, true_item in zip(clues, generated_answers, true_data):
#         true_answer = true_item["answer"]
#         print(f"Clue: {clue}")
#         print(f"Generated Answer: {gen_answer}")
#         print(f"True Answer: {true_answer}")
#         print(f"Correct: {gen_answer.strip().lower() == true_answer.strip().lower()}")
#         print()

# # Evaluate on training data
# # evaluate(train_clues, train_answers, train_data)

# # Evaluate on test data
# evaluate(test_clues, test_answers, test_data)

# # Evaluate on evaluation data
# evaluate(eval_clues, eval_answers, eval_data)
